# Azure AI Foundry

<center><img src="../../../images/Azure-AI-Foundry_1600x900.jpg" alt="Azure AI Foundry" width="600">

## Laboratory 4

In this laboratory we will explore advanced frameworks for building intelligent applications: **Semantic Kernel** and **AutoGen** with Azure OpenAI. We will learn how to create conversational agents, orchestrate multiple agents, implement custom plugins, and develop complex automation workflows.

**Semantic Kernel** is an open-source SDK from Microsoft that allows integrating AI models (like Azure OpenAI) with conventional programming languages, offering functionalities like plugins, planning, and memory.

**AutoGen** is a Microsoft framework for creating multi-agent applications where different agents can collaborate to solve complex problems through structured conversation.

### Prerequisites

Make sure environment variables are configured in the `.env` file at the repository root with your Azure OpenAI credentials.

### Exercise 1 - Initial Setup and Semantic Kernel

Let's start by installing and importing the necessary libraries to work with Semantic Kernel and AutoGen.

In [ ]:
# Installing necessary dependencies
%pip install semantic-kernel autogen-agentchat autogen-ext[openai,azure] python-dotenv

In [ ]:
import os
import asyncio
from dotenv import load_dotenv
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.core_plugins import TextPlugin
from semantic_kernel.functions import KernelArguments
from semantic_kernel.prompt_template import PromptTemplateConfig

# Imports for AutoGen version 0.6+
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Loading environment variables
load_dotenv(dotenv_path="../../../.env")

In [ ]:
# Azure OpenAI credentials configuration
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")

print("Settings loaded:")
print(f"Endpoint: {azure_endpoint}")
print(f"Deployment: {deployment_name}")
print(f"API Version: {api_version}")

#### Configuring Semantic Kernel

Semantic Kernel acts as an orchestration layer that allows combining AI models with conventional code. Let's create a kernel and configure it to use Azure OpenAI:

In [ ]:
# Initializing Semantic Kernel
kernel = sk.Kernel()

# Adding Azure OpenAI chat service
chat_service = AzureChatCompletion(
    deployment_name=deployment_name,
    endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

kernel.add_service(chat_service)

print("Semantic Kernel configured successfully!")

#### First Example: Simple Function with Semantic Kernel

Let's create a simple function that uses Semantic Kernel to generate content:

In [ ]:
# Creating a simple function using prompt template
prompt_template = """
You are an expert in {{$topic}}. 
Create an informative and concise summary about {{$topic}} in {{$language}}.
The summary should be approximately {{$length}} words.

Topic: {{$topic}}
"""

# Creating the kernel function
summary_function = kernel.add_function(
    function_name="CreateSummary",
    plugin_name="ContentPlugin",
    prompt=prompt_template,
    description="Creates a summary about any topic"
)

# Executing the function
async def run_summary_example():
    arguments = KernelArguments(
        topic="Artificial Intelligence",
        language="English", 
        length="150"
    )
    
    result = await kernel.invoke(summary_function, arguments)
    return result

# Running the example
result = await run_summary_example()
print("Summary generated by Semantic Kernel:")
print(result)

### Exercise 2 - Plugins and Advanced Features

#### Working with Built-in Plugins

Semantic Kernel offers integrated plugins that provide common functionalities. Let's explore the TextPlugin:

In [ ]:
# Adding built-in text plugin
kernel.add_plugin(TextPlugin(), plugin_name="text")

# Example of using text plugin
text_to_process = """
Azure AI Foundry is a complete platform for developing AI applications. 
It offers tools for training, deploying, and managing machine learning models. 
With native integration to Azure OpenAI, developers can create intelligent applications quickly.
"""

async def demonstrate_text_plugin():
    # Function to make uppercase
    uppercase_result = await kernel.invoke(
        plugin_name="text",
        function_name="uppercase", 
        input=text_to_process
    )
    
    print("Text in uppercase:")
    print(uppercase_result)
    print("\n" + "="*50 + "\n")
    

await demonstrate_text_plugin()

#### Creating a Custom Plugin

Let's create a custom plugin that simulates mathematical operations and data analysis:

In [ ]:
from semantic_kernel.functions import kernel_function
from typing import Annotated
import statistics

class MathPlugin:
    """Custom plugin for mathematical operations"""
    
    @kernel_function(
        description="Calculates the average of a list of numbers",
        name="calculate_average"
    )
    def calculate_average(
        self, 
        numbers: Annotated[str, "List of comma-separated numbers"]
    ) -> Annotated[str, "The average of the numbers"]:
        try:
            num_list = [float(x.strip()) for x in numbers.split(',')]
            average = statistics.mean(num_list)
            return f"The average of numbers {numbers} is: {average:.2f}"
        except Exception as e:
            return f"Error calculating average: {str(e)}"
    
    @kernel_function(
        description="Finds the maximum number in a list",
        name="find_maximum"
    )
    def find_maximum(
        self, 
        numbers: Annotated[str, "List of comma-separated numbers"]
    ) -> Annotated[str, "The maximum number"]:
        try:
            num_list = [float(x.strip()) for x in numbers.split(',')]
            maximum = max(num_list)
            return f"The maximum number in {numbers} is: {maximum}"
        except Exception as e:
            return f"Error finding maximum: {str(e)}"

# Adding the custom plugin to the kernel
math_plugin = MathPlugin()
kernel.add_plugin(math_plugin, plugin_name="math")

# Testing the custom plugin
async def test_custom_plugin():
    numbers = "10, 25, 15, 30, 8, 22"
    
    # Calculating average
    avg_result = await kernel.invoke(
        plugin_name="math",
        function_name="calculate_average",
        numbers=numbers
    )
    print(avg_result)
    
    # Finding maximum
    max_result = await kernel.invoke(
        plugin_name="math", 
        function_name="find_maximum",
        numbers=numbers
    )
    print(max_result)

await test_custom_plugin()

### Exercise 3 - Introduction to AutoGen

**AutoGen** is a powerful framework for creating multi-agent systems where different agents can collaborate through structured conversation. Unlike Semantic Kernel which focuses on function orchestration, AutoGen allows creating specialized agents that can debate, collaborate, and reach conclusions autonomously.

#### AutoGen Fundamental Concepts

- **Agents**: Entities that can send and receive messages
- **Conversation**: Message flow between agents
- **Roles**: Specific agent roles (Assistant, User Proxy, etc.)
- **Group Chat**: Conversations between multiple agents
- **Termination**: Conditions to end conversations

#### Basic AutoGen Configuration

**Important note**: This example uses the new AutoGen API (version 0.6+) which replaces `llm_config` with `model_client`. Make sure to use `autogen-ext[openai,azure]` to have access to necessary extensions.

Let's start by configuring basic agents and creating a simple conversation:

In [ ]:
# Basic AutoGen configuration using the new API (version 0.6+)
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# Creating Azure OpenAI client
azure_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=deployment_name,
    model=deployment_name,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

# Creating a specialized assistant agent using the new API
creative_assistant = AssistantAgent(
    name="creative_assistant",
    model_client=azure_model_client,
    system_message="""You are a creative assistant specialized in idea generation.
    Your function is to help with brainstorming and create innovative content.
    Be creative, but keep your responses concise and practical."""
)

print("AutoGen agents configured successfully!")
print(f"Assistant: {creative_assistant.name}")
print("Model configured to use Azure OpenAI")

#### Simple Example: Conversation Between Agents

Let's create a simple conversation between the user and the creative assistant to generate ideas for a project:

In [ ]:
# Simple conversation example between agents using the new API
async def execute_autogen_example():
    """
    Simple example of how AutoGen agents can collaborate
    to generate creative ideas for a project
    """
    
    # Initial user message
    initial_message = """
    I need creative ideas for a mobile app that helps people 
    organize their daily routine. The app should be innovative and easy to use.
    
    Please give me 3 main ideas with a brief description of each one.
    """
    
    # Executing the agent
    try:
        print("=== Starting Conversation with AutoGen ===\n")
        
        # Using the new API - run method
        result = await creative_assistant.run(task=initial_message)
        
        print("Assistant's response:")
        # The new API returns a TaskResult with messages
        if result.messages:
            for message in result.messages:
                if hasattr(message, 'content'):
                    print(f"- {message.content}")
        
        print("\n=== Conversation Finished ===")
        return result
        
    except Exception as e:
        print(f"Error during conversation: {str(e)}")
        return None

# Running the example
conversation_result = await execute_autogen_example()

#### Understanding What Happened

In the example above, we demonstrated the fundamental concepts of AutoGen in version 0.6+:

1. **Agent Configuration**: We created an agent using the new API:
   - `AssistantAgent`: Specialized in creativity and idea generation
   - Uses `model_client` instead of the old `llm_config`

2. **`run()` Method**: The new API uses the `run()` method which:
   - Accepts a task as string
   - Returns a `TaskResult` with messages
   - Is asynchronous by default

3. **Specialization**: The agent has a `system_message` that defines its role and specific behavior.

4. **Simplicity**: The new API is cleaner and focused on specific use cases.

### Best Practices and Advanced Use Cases

#### When to Use Semantic Kernel vs AutoGen

**Semantic Kernel is ideal for:**
- AI orchestration with traditional code
- Creating reusable plugins and functions
- Applications that need automatic planning
- Integration with existing systems
- Single-agent applications with complex functionalities

**AutoGen is ideal for:**
- Complex multi-agent systems
- Discussion and brainstorming simulations
- Workflows involving multiple perspectives
- Collaborative process automation
- Distributed decision-making systems

#### Practical Use Cases

1. **Customer Service System**
   - Semantic Kernel: Creating plugins for order queries, products, etc.
   - AutoGen: Specialized agents (triage, sales, technical support)

2. **Corporate Document Analysis**
   - Semantic Kernel: Functions for text extraction and processing
   - AutoGen: Reviewer agents, analysts, and domain specialists

3. **Project Planning**
   - Semantic Kernel: Plugins for schedule and resource calculations
   - AutoGen: Agents representing different stakeholders

4. **Recommendation System**
   - Semantic Kernel: Filtering and similarity calculation functions
   - AutoGen: Agents with different recommendation criteria

In [ ]:
# Final Example: Product Analysis System
# Combining SK and AutoGen (version 0.6+) for product feedback analysis

# SK plugin for sentiment analysis (simulated)
class SentimentPlugin:
    @kernel_function(
        description="Analyzes the sentiment of a text",
        name="analyze_sentiment"
    )
    def analyze_sentiment(
        self, 
        text: Annotated[str, "Text for sentiment analysis"]
    ) -> Annotated[str, "Sentiment analysis result"]:
        # Simple sentiment analysis simulation
        positive_words = ["excellent", "great", "good", "recommend", "perfect"]
        negative_words = ["bad", "terrible", "horrible", "don't recommend", "defect"]
        
        text_lower = text.lower()
        positive_score = sum(1 for word in positive_words if word in text_lower)
        negative_score = sum(1 for word in negative_words if word in text_lower)
        
        if positive_score > negative_score:
            return f"POSITIVE (Score: +{positive_score - negative_score})"
        elif negative_score > positive_score:
            return f"NEGATIVE (Score: -{negative_score - positive_score})"
        else:
            return "NEUTRAL (Score: 0)"

# Adding the plugin to the kernel
sentiment_plugin = SentimentPlugin()
kernel.add_plugin(sentiment_plugin, plugin_name="sentiment")

# Creating specialized agents for product analysis using the new API
quality_agent = AssistantAgent(
    name="quality_specialist",
    model_client=azure_model_client,
    system_message="""You are a product quality specialist.
    Analyze feedback focusing on quality aspects, durability, and design.
    Identify patterns and suggest improvements."""
)

experience_agent = AssistantAgent(
    name="ux_specialist",
    model_client=azure_model_client,
    system_message="""You are a user experience specialist.
    Analyze feedback focusing on usability, ease of use, and satisfaction.
    Suggest improvements in customer experience."""
)

# Example of complete system execution
async def product_analysis_system():
    feedbacks = [
        "The product is excellent! Very easy to use and the quality was surprising.",
        "I had problems with the product. It doesn't work as expected, terrible experience.",
        "The design is good but could improve usability. I recommend with reservations."
    ]
    
    print("=== Product Analysis System ===\n")
    
    # Sentiment analysis using Semantic Kernel
    sentiment_analyses = []
    for i, feedback in enumerate(feedbacks, 1):
        sentiment = await kernel.invoke(
            plugin_name="sentiment",
            function_name="analyze_sentiment",
            text=feedback
        )
        sentiment_analyses.append(f"Feedback {i}: {sentiment}")
        print(f"Feedback {i}: {feedback}")
        print(f"Sentiment: {sentiment}\n")
    
    # Preparing consolidated report
    consolidated_report = f"""
    Product Feedback Analysis - Consolidated Report
    
    Analyzed feedbacks:
    {chr(10).join(f'{i+1}. {feedback}' for i, feedback in enumerate(feedbacks))}
    
    Sentiment analyses:
    {chr(10).join(sentiment_analyses)}
    
    Please provide your specialized analysis and recommendations to improve the product.
    """
    
    # Quality specialist analysis
    print("=== Quality Specialist Analysis ===")
    quality_result = await quality_agent.run(task=consolidated_report)
    if quality_result.messages:
        print(quality_result.messages[-1].content)
    
    print("\n=== UX Specialist Analysis ===")
    ux_result = await experience_agent.run(task=consolidated_report)
    if ux_result.messages:
        print(ux_result.messages[-1].content)

# Run the system (uncomment to test)
# await product_analysis_system()

print("Product analysis system configured!")
print("Uncomment the line 'await product_analysis_system()' to run the complete example.")

### Conclusion and Next Steps

In this laboratory, we explored the powerful capabilities of **Semantic Kernel** and **AutoGen** (version 0.6+) to create advanced intelligent applications with Azure OpenAI. We learned:

#### Semantic Kernel
- ✅ Kernel configuration and initialization
- ✅ Creation and use of custom functions
- ✅ Working with built-in and custom plugins
- ✅ Prompt templates and dynamic arguments

#### AutoGen
- ✅ Agent configuration with the new API
- ✅ Using `model_client` instead of `llm_config`
- ✅ Asynchronous `run()` method for execution
- ✅ Domain-specific agent specialization

### Additional Resources

To deepen your knowledge, explore the following resources:

#### Semantic Kernel
- [Official Semantic Kernel Documentation](https://learn.microsoft.com/en-us/semantic-kernel/)
- [Semantic Kernel GitHub Repository](https://github.com/microsoft/semantic-kernel)
- [Examples and Tutorials](https://learn.microsoft.com/en-us/semantic-kernel/get-started/)
- [Community Plugins](https://github.com/microsoft/semantic-kernel/tree/main/python/semantic_kernel/core_plugins)

#### AutoGen
- [Official AutoGen Documentation](https://microsoft.github.io/autogen/)
- [AutoGen GitHub Repository](https://github.com/microsoft/autogen)
- [Example Gallery](https://microsoft.github.io/autogen/docs/Examples)
- [Tutorial Notebooks](https://github.com/microsoft/autogen/tree/main/notebook)

#### Azure AI Foundry
- [Azure AI Foundry Portal](https://ai.azure.com/)
- [Azure OpenAI Documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/)
- [Available Models](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/foundry-models-overview)
- [Best Practices](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/best-practices)

### Practice Challenges

1. **Product System Expansion**: Add more plugins to Semantic Kernel for product image analysis
2. **Specialized Agents**: Create agents for different sectors (finance, administrative, education)
3. **Complex Workflow**: Implement a multi-stage approval system using AutoGen
4. **User Interface**: Create a web interface to interact with your agents